In [1]:
# inverse kin

import sympy as smp

# maybe we choose the signs base on which is closest? but also the one within possible range idk

px, py, L1, L2 = smp.symbols('px, py, L1, L2')

# defaults: sign1theta2 = 1  ;  sign2theta2 = 1
theta2 = smp.acos( (px**2 + py**2 - L1**2 - L2**2) / (2*L1*L2) )
theta2_f = smp.lambdify( (px, py, L1, L2) , theta2)


costh2, sinth2 = smp.symbols('costh2, sinth2')

theta1 = smp.atan(py/px) - smp.atan(L2*sinth2/(L1+L2*costh2))
theta1_f = smp.lambdify( (px, py, L1, L2, costh2, sinth2) , theta1)

_, _, px, py, L1, L2 = 1, 1, 244-200, 189-200, 70, 70
(px**2 + py**2 - L1**2 - L2**2) /(2*L1*L2)

#theta2_f(1, 1, 244, 189, 70, 70)


-0.7901020408163265

In [3]:
# file:///C:/Users/Kream/Desktop/SmartImpedanceControl/ResearchPapers/3.pdf
import cv2
import numpy as np

# physics info
L1 = 70
L2 = 70

# window size 
width = 400
height = 400

center = (int(width/2), int(height/2))
theta1 = np.pi*1/4
theta2 = np.pi*1/4
motor1_angle = int(theta1*180/np.pi)
motor2_angle = int(theta2*180/np.pi)

# debugging
print("°", motor1_angle, ", °", motor2_angle)


def inverseKin(px, py):
    
    # we gotta offset desired from the center
    px -= center[0]
    py -= center[1]
    tmp_px = px
    px = py
    py = tmp_px
    theta2 = theta2_f(px, py, L1, L2)
    theta1 = theta1_f(px, py, L1, L2, np.cos(theta2), np.sin(theta2))
    print("th1:", theta1, "th2:", theta2)
    return theta1, theta2

def forwardKin(theta1, theta2):
    x = L2*np.sin(theta1 + theta2) + L1*np.sin(theta1)
    y = L2*np.cos(theta1 + theta2) + L1*np.cos(theta1)
    return list((int(x) + center[0], int(y) + center[1]))

def forwardElbowKin(theta1):
    x = L1*np.sin(theta1)
    y = L1*np.cos(theta1)
    return list((int(x) + center[0], int(y) + center[1]))

def desiredIntroduced(event, x, y, flags, param):
    global desired, theta1, theta2, end_effector, elbow
    if event==cv2.EVENT_LBUTTONDOWN:
        desired = [x, y]
        theta1, theta2 = inverseKin(x, y) # we don't need them in the simulation but we need them in the physical robot
        end_effector = forwardKin(theta1, theta2)
        elbow = forwardElbowKin(theta1)

end_effector = forwardKin(theta1, theta2)
elbow = forwardElbowKin(theta1)
desired = end_effector.copy()

# allow the user to edit the desired point real time
cv2.destroyAllWindows()
cv2.namedWindow('output', cv2.WINDOW_AUTOSIZE)
cv2.setMouseCallback('output', desiredIntroduced)
    
while True:
    window = np.ones((height, width, 3), dtype=np.uint8) #*255

    window = cv2.line(window, tuple(center), tuple(elbow), (255, 0, 0), 6)
    window = cv2.line(window, tuple(elbow), tuple(end_effector), (255, 0, 0), 6)
    
    window = cv2.circle(window, tuple(desired), 5, (255, 255, 255), 2)   # we have casual coordinates but we want zero to be the middle of screen with a range of -4 to 4 just bcz it works with our other values
        
    cv2.imshow('output', window)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

° 45 , ° 45
th1: 0.40318255007054427 th2: 1.7480495786312054
th1: 0.17196358120037747 th2: 2.1735812480639716
th1: 0.0610629615284739 th2: 2.205525159795303
th1: -0.12820601159033462 th2: 2.213149969906705
th1: -0.16911949379828295 th2: 1.926733967423978
th1: -0.056717758189752776 th2: 1.5973300523913405
th1: 0.03894771337778402 th2: 1.387734776817445
th1: 0.0522365999967882 th2: 1.2422652019493443
th1: -0.0939779119376799 th2: 1.2327626873084172
th1: -0.1971438219461652 th2: 1.677016363709237
th1: -0.2597765111623227 th2: 1.854165292281354
th1: -0.40845586516228727 th2: 2.054200432174069
th1: -0.5024730455516432 th2: 2.1640740616899325
th1: -0.5548128970062588 th2: 2.3051425771986818
th1: -0.5559762196093224 th2: 2.4233436917017173
th1: -0.47567466972164285 th2: 2.461558146400632
th1: -0.579712473837386 th2: 2.555230592864886
th1: -1.255956675500266 th2: 2.5619029382383722
th1: -1.4048062898270692 th2: 2.5258184704458104
th1: -1.5844699065380106 th2: 2.339926063906449
th1: -1.68078226

<lambdifygenerated-1>:2: RuntimeWarning: invalid value encountered in arccos
  return arccos((1/2)*(-L1**2 - L2**2 + px**2 + py**2)/(L1*L2))


th1: nan th2: nan


ValueError: cannot convert float NaN to integer

th1: -1.4102606866540164 th2: 0.9530693782696065
th1: -1.3309463383367317 th2: 1.0560122297637617
th1: nan th2: nan


...

th1: nan th2: nan


ValueError: cannot convert float NaN to integer

th1: nan th2: nan


ValueError: cannot convert float NaN to integer

th1: -0.9242658732140546 th2: 0.1661761375794208
th1: 0.4459782289814437 th2: 1.8834145611019377
th1: -2.3034361583285174 th2: 2.3754812802036307
th1: -2.0202439172667104 th2: 2.1498652614220854
th1: -2.1269922073748697 th2: 2.1619833044601613
th1: -2.144893617327688 th2: 1.8518279399569555
th1: -2.135122896190868 th2: 1.7721542475852274
th1: -2.161523302322196 th2: 1.9912375236247652
th1: -2.0241071544363116 th2: 2.1232716725199765
th1: -1.7021311755189032 th2: 2.3173071616117196
th1: 0.024918905777910005 th2: 1.9884448769768794
th1: 0.28244711026313585 th2: 1.8156856788387916
th1: 0.45182552404953524 th2: 1.6754772016541324
th1: 0.6141946907222956 th2: 1.4186797437796814
th1: 0.6313327577967921 th2: 1.13363733029655
th1: 0.4120277280571727 th2: 1.1893697553761338
th1: -0.14258993241364604 th2: 1.6746564172206986
th1: -0.8470067267847924 th2: 2.207046678858232
th1: -1.372896408856019 th2: 2.3658162418746063
th1: -1.6109908851590322 th2: 2.3269418200037246
th1: -1.6285260537531867 th2: